In [6]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# TEST = r'C:\Users\arish\open-source\psi-detection\data\intermediate\H.sapiens100\test.csv'
TRAIN = r'C:\Users\arish\open-source\psi-detection\data\intermediate\hsapiens495\train.csv'
# VALID = r'C:\Users\arish\open-source\psi-detection\data\intermediate\H.sapiens100\valid.csv'

In [29]:
def space_sequence(dataframe: pd.DataFrame):
    dataframe['x'] = dataframe['x'].map(list).map(lambda x: ' '.join(x))


def prepare_single_sequence(sequence: str):
    return ' '.join(list(sequence))


def split_dataframe(dataframe: pd.DataFrame):
    return dataframe['x'].values, dataframe['y'].values

In [4]:
train_data = pd.read_csv(TRAIN, header=None)
train_data.columns = ['x', 'y']
space_sequence(train_data)

# valid_data = pd.read_csv(VALID, header=None)
# valid_data.columns = ['x', 'y']
# space_sequence(valid_data)
#
# test_data = pd.read_csv(TEST, header=None)
# test_data.columns = ['x', 'y']
# space_sequence(test_data)

# test_data = split_dataframe(test_data)
train_data = split_dataframe(train_data)
# valid_data = split_dataframe(valid_data)

In [7]:
train_x, test_x, train_y, test_y = train_test_split(train_data[0], train_data[1], test_size=0.2, random_state=1)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.25, random_state=1)

In [8]:
VOCAB_SIZE = 4 + 2
BATCH_SIZE = 64

In [9]:
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_x)

In [10]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64,
        mask_zero=False,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [11]:
model.predict(train_x)

19/19 [==============================] - 1s 5ms/step


array([[0.50359607],
       [0.50221145],
       [0.50271153],
       [0.49999255],
       [0.498415  ],
       [0.5003724 ],
       [0.49881646],
       [0.49936706],
       [0.5014425 ],
       [0.5016416 ],
       [0.5017632 ],
       [0.49708444],
       [0.50024813],
       [0.5007146 ],
       [0.5019153 ],
       [0.5020216 ],
       [0.50063384],
       [0.49884403],
       [0.49914393],
       [0.5004883 ],
       [0.50326264],
       [0.50090206],
       [0.49963796],
       [0.49692494],
       [0.50263554],
       [0.4975864 ],
       [0.49873704],
       [0.5000389 ],
       [0.4985265 ],
       [0.5011708 ],
       [0.50138223],
       [0.49898332],
       [0.5017166 ],
       [0.49883205],
       [0.5008302 ],
       [0.50166404],
       [0.4997287 ],
       [0.50159377],
       [0.49831915],
       [0.5010004 ],
       [0.500267  ],
       [0.49954024],
       [0.49880388],
       [0.5011046 ],
       [0.49339545],
       [0.50071913],
       [0.5020506 ],
       [0.502

In [12]:
model.compile(
    loss=tf.keras.losses.MSE,
    optimizer='adam',
    metrics=['accuracy']
)

In [15]:
model.fit(
    train_data[0],
    train_data[1],
    epochs=20,
    validation_data=(valid_x, valid_y)
)

Epoch 1/20
31/31 [==============================] - 1s 17ms/step - loss: 0.2186 - accuracy: 0.6364 - val_loss: 0.2019 - val_accuracy: 0.6818
Epoch 2/20
31/31 [==============================] - 0s 14ms/step - loss: 0.2113 - accuracy: 0.6586 - val_loss: 0.1897 - val_accuracy: 0.7323
Epoch 3/20
31/31 [==============================] - 0s 15ms/step - loss: 0.2125 - accuracy: 0.6465 - val_loss: 0.2153 - val_accuracy: 0.6414
Epoch 4/20
31/31 [==============================] - 0s 15ms/step - loss: 0.2063 - accuracy: 0.6677 - val_loss: 0.1801 - val_accuracy: 0.7273
Epoch 5/20
31/31 [==============================] - 0s 15ms/step - loss: 0.1970 - accuracy: 0.6848 - val_loss: 0.1739 - val_accuracy: 0.7576
Epoch 6/20
31/31 [==============================] - 0s 15ms/step - loss: 0.1928 - accuracy: 0.6879 - val_loss: 0.1598 - val_accuracy: 0.7727
Epoch 7/20
31/31 [==============================] - 0s 14ms/step - loss: 0.1803 - accuracy: 0.7232 - val_loss: 0.1483 - val_accuracy: 0.8182
Epoch 8/20
31

In [16]:
model.evaluate(test_x, test_y)

7/7 [==============================] - 0s 6ms/step - loss: 0.0348 - accuracy: 0.9646


[0.034801334142684937, 0.9646464586257935]

In [23]:
test_x[1], test_y[1]

('C A G G C A G A C A U G G U U U G G A A A', 0)

In [31]:
model.predict([prepare_single_sequence('UUUUCAUCACUAUGGCUUAGC')])

1/1 [==============================] - 0s 44ms/step


array([[0.00431074]], dtype=float32)